In [1]:
### transform contours
import numpy as np
from numpy import asarray
import cv2
import math
import os.path, sys
from PIL import Image, ImageOps
from matplotlib import pyplot as plt
import xml.etree.ElementTree as ET
import os
from lxml import etree as et
from matplotlib import pyplot as plt
import pandas as pd


In [2]:
def cart2pol(x, y):
    theta = np.arctan2(y, x)
    rho = np.hypot(x, y)
    return theta, rho


def pol2cart(theta, rho):
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y


def rotate_contour(cnt, angle):
    #print('cnt', cnt)
    M = cv2.moments(cnt)
    try:
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    except:
        cx = int(M['m10']/1)
        cy = int(M['m01']/1)
        
    cnt_norm = cnt - [cx, cy]
    
    coordinates = cnt_norm[:, 0, :]
    
    #print('coordinates', coordinates)
    xs, ys = coordinates[:, 0], coordinates[:, 1]
    thetas, rhos = cart2pol(xs, ys)
    
    thetas = np.rad2deg(thetas)
    thetas = (thetas + angle) % 360
    thetas = np.deg2rad(thetas)
    
    xs, ys = pol2cart(thetas, rhos)
    
    cnt_norm[:, 0, 0] = xs
    cnt_norm[:, 0, 1] = ys

    cnt_rotated = cnt_norm + [cx, cy]
    cnt_rotated = cnt_rotated.astype(np.int32)

    return cnt_rotated

In [3]:
def find_contour_center(img_path):
    im = cv2.imread(img_path)
    #print(img_path)
    img_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)   
    ret, thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret = ret - 12
    ret, thresh = cv2.threshold(img_gray, ret, 255 ,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    try:
        cnt_rotated = rotate_contour(contours[0], 0)
        M = cv2.moments(cnt_rotated)
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        center_0 = (cx, cy)
    except:
        cnt_rotated = rotate_contour(contours[1], 0)
        M = cv2.moments(cnt_rotated)
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        center_0 = (cx, cy)
    return contours, center_0

def find_contour_center_1(img_path):
    im = cv2.imread(img_path)
    img_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)   
    ret, thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret = ret - 12
    ret, thresh = cv2.threshold(img_gray, ret, 255 ,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    image = Image.open(img_path)
    width, height =image.size
    center_x = int(width/2)
    center_y = int(height/2)
    center_0 = (center_x, center_y)
    return contours, center_0


def rotate_from_origin(img_path, angle):
    im = cv2.imread(img_path)
    
    img_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)   
    ret, thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    ret = ret - 12
    ret, thresh = cv2.threshold(img_gray, ret, 255 ,cv2.THRESH_BINARY)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)   
    
    image = Image.open(img_path)
    width, height =image.size
    center_1a = (int(width/2), int(height/2))


    try:
        cnt_rotated = rotate_contour(contours[0], angle)
        M = cv2.moments(cnt_rotated)
        if M['m00'] == 0:
            M['m00'] = 1
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    except:
        cnt_rotated = rotate_contour(contours[1], angle)
        M = cv2.moments(cnt_rotated)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    
    center_1 = (cx, cy)
    
    return contours, cnt_rotated, center_1


def rotate_from_origin_1(img_path, angle):
    im = cv2.imread(img_path)
    img_gray = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)   
    ret, thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    image = Image.open(img_path)
    width, height =image.size
    center_1a = (int(width/2), int(height/2))

    try:
        cnt_rotated = rotate_contour(contours[0], angle)
        M = cv2.moments(cnt_rotated)
        if M['m00'] == 0:
            M['m00'] = 1
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    except:
        cnt_rotated = rotate_contour(contours[1], angle)
        M = cv2.moments(cnt_rotated)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
    center_1 = (cx, cy) 
    return contours, cnt_rotated, center_1


def get_move_distance(center_1, center_2):
    distance_x = center_2[0] - center_1[0] 
    distance_y = center_2[1] - center_1[1]
    return distance_x, distance_y

def move_contour_get_bbox(cnt_rotated, distance_x, distance_y):
    cnt_rotated_1 = []
    for i in range(len(cnt_rotated)):
        x, y = cnt_rotated[i][0][0], cnt_rotated[i][0][1]
        x_1 = x+distance_x
        y_1 = y+distance_y
        cnt = [[x_1, y_1]]
        cnt_rotated_1.append(cnt)
    cnt_rotated_move = np.array(cnt_rotated_1)
    x,y,w,h = cv2.boundingRect(cnt_rotated_move)
    xmin = x
    ymin = y
    xmax = x + w
    ymax = y + h
    return cnt_rotated_move, xmin, ymin, xmax, ymax

In [4]:
#create dataframe 

def create_object_df(obj_path_list, label):
    grain_size_lst = []
    grain_path_lst = []
    contours_lst = []
    center_0_lst = []
    label_lst = [label] * len(obj_path_list)
    for i in range(len(obj_path_list)):
        grain_path = obj_path_list[i]
        basename = os.path.basename(grain_path)
        grain_path_lst.append(basename)
        image = Image.open(obj_path_list[i])
        width, height = image.size
        grain_size = (width, height)
        grain_size_lst.append(grain_size)
        contours, center_0 = find_contour_center_1(grain_path)
        contours_lst.append(contours)
        center_0_lst.append(center_0)
        
    df = pd.DataFrame(list(zip(label_lst, grain_path_lst, grain_size_lst, contours_lst, center_0_lst)),
               columns =['label', 'path', 'size', 'contour', 'center_0'])
    return df
    

In [5]:
def create_xml(directory, imgfilepath, object_list, size_tuple, savedir):
    """
    params:
    - imgfilepath: path of corresponding img file. only the basename will be actually used.
    - object_list: python list of element in tuple of (classname, [x1,y1,x2,y2])
    - savedir: output directory to save generated xml file
    """
    basename = os.path.basename(imgfilepath)
    filename, _ = os.path.splitext(basename)
    root = et.Element('annotation')
    
    fn_elem = et.SubElement(root, 'filename')
    bn = os.path.basename(imgfilepath)
    fn, _ = os.path.splitext(bn)
    img_bn = f"{fn}.jpg"
    fn_elem.text = img_bn

    path = et.SubElement(root, 'path')
    path.text = directory + imgfilepath

    source = et.SubElement(root, 'source')
    database = et.SubElement(source, 'database')
    database.text = 'Unknown'

    size = et.SubElement(root, 'size')
    width = et.SubElement(size, 'width')
    width.text = str(size_tuple[0])
    height = et.SubElement(size, 'height')
    height.text = str(size_tuple[1])
    depth = et.SubElement(size, 'depth')
    depth.text = str(size_tuple[2])
    
    for classname, (x1,y1,x2,y2) in object_list:
        object_elem = et.SubElement(root, 'object')
        name = et.SubElement(object_elem, 'name')
        name.text = classname
        bndbox = et.SubElement(object_elem, 'bndbox')
        xmin = et.SubElement(bndbox, 'xmin')
        xmin.text = str(x1)
        ymin = et.SubElement(bndbox, 'ymin')
        ymin.text = str(y1)
        xmax = et.SubElement(bndbox, 'xmax')
        xmax.text = str(x2)
        ymax = et.SubElement(bndbox, 'ymax')
        ymax.text = str(y2)
    out = et.tostring(root, pretty_print=True, encoding='utf8') # if element attributes contains some non ascii characters, then need to specify encoding. if not the case, then encoding doesn't need to be set
    savepath = os.path.join(savedir, f'{fn}.xml')
    with open(savepath, 'wb') as fd:
        fd.write(out)

def create_xml_from_dir(dir, object_list_list, savedir):
    dirs = os.listdir(dir) #select jpg files only from directory
    jpg_dirs = []
    for i in range(len(dirs)):
        if '.jpg' in dirs[i]:
            jpg_dirs.append(dirs[i])
    print(len(jpg_dirs), len(object_list_list))

    for i in range(len(jpg_dirs)): #generates jpg file to XML
        if len(jpg_dirs) == len(object_list_list):
            imgfilepath = jpg_dirs[i]
            object_list = object_list_list[i]
            create_xml(dir, imgfilepath, object_list, size_tuple, savedir)
        else:
            print('dir contains other files than jpg')

In [6]:
path = '/Users/malha/desktop/reimage/pic'

item_list = os.listdir(path)
item_list = item_list[1:]

folder_list = []
for item in item_list:
    folder = os.path.join(path, item)
    folder_list.append(folder)
    
####folder_list

broken_folder = '/Users/malha/desktop/reimage/pic/Broken'
damaged_folder = '/Users/malha/desktop/reimage/pic/Damaged'
foreign_folder = '/Users/malha/desktop/reimage/pic/Foreign'
green_folder = '/Users/malha/desktop/reimage/pic/Green'
normal_folder = '/Users/malha/desktop/reimage/pic/Normal'
# twigs_folder = '/Users/charissayu/desktop/reimage (3)/pic/twigs'

def get_obj_path(obj_folder):
    obj_list = os.listdir(obj_folder)
    obj_folder_list = []
    for i in range(len(obj_list)):
        obj = os.path.join(obj_folder, obj_list[i])
        obj_folder_list.append(obj)
    return obj_folder_list
    

In [7]:
def remove_non_png_file(obj_path_list):
    pth_list_1 = []
    for path in obj_path_list:
        if '.png' in path:
            pth_list_1.append(path)
    return pth_list_1

In [8]:
broken_pth_list =  get_obj_path(broken_folder)
damaged_pth_list =  get_obj_path(damaged_folder)
foreign_pth_list =  get_obj_path(foreign_folder)
green_pth_list =  get_obj_path(green_folder)
normal_pth_list =  get_obj_path(normal_folder)
# twigs_pth_list =  get_obj_path(twigs_folder)

broken_pth_list = remove_non_png_file(broken_pth_list)
damaged_pth_list = remove_non_png_file(damaged_pth_list)
foreign_pth_list = remove_non_png_file(foreign_pth_list)
green_pth_list = remove_non_png_file(green_pth_list)
normal_pth_list = remove_non_png_file(normal_pth_list)
# twigs_pth_list = remove_non_png_file(twigs_pth_list)

In [9]:
df_broken = create_object_df(broken_pth_list, 'broken')
df_damaged = create_object_df(damaged_pth_list, 'damaged')
df_foreign = create_object_df(foreign_pth_list, 'foreign')
df_green = create_object_df(green_pth_list, 'green')
df_normal = create_object_df(normal_pth_list, 'normal')
# df_twigs = create_object_df(twigs_pth_list, 'twigs')


In [10]:
df = pd.concat([df_broken, df_damaged, df_foreign, df_green, df_normal])
df

,label,path,size,contour,center_0
0,broken,Broken_0.png,"(86, 86)","([[[31 0]], [[30 1]], [[29 1]], [[28 2]], ...","(43, 43)"
1,broken,Broken_1.png,"(104, 83)","([[[54 0]], [[53 1]], [[48 1]], [[47 2]], ...","(52, 41)"
2,broken,Broken_10.png,"(112, 93)","([[[ 5 76]]], [[[111 11]]], [[[46 0]], [[45 ...","(56, 46)"
3,broken,Broken_100.png,"(84, 95)","([[[35 1]], [[34 2]], [[32 2]], [[31 3]], ...","(42, 47)"
4,broken,Broken_101.png,"(96, 83)","([[[40 0]], [[38 2]], [[37 2]], [[36 3]], ...","(48, 41)"
...,...,...,...,...,...
145,normal,Normal_95.png,"(117, 84)","([[[49 0]], [[48 1]], [[44 1]], [[43 2]], ...","(58, 42)"
146,normal,Normal_96.png,"(88, 81)","([[[42 0]], [[41 1]], [[37 1]], [[36 2]], ...","(44, 40)"
147,normal,Normal_97.png,"(87, 99)","([[[36 0]], [[35 1]], [[31 1]], [[30 2]], ...","(43, 49)"
148,normal,Normal_98.png,"(106, 66)","([[[48 0]], [[47 1]], [[43 1]], [[42 2]], ...","(53, 33)"


In [11]:
def get_center_label_size(textfile):
    a_file = open(textfile, "r")
    list_of_lists = []
    for line in a_file:
        stripped_line = line.strip()
        line_list = stripped_line.split()
        list_of_lists.append(line_list)
    
    final_list = []
    for lst in list_of_lists:
        if len(lst) == 7:
            final_list.append(lst)
        
    final_lst = []
    for i in range(len(final_list)):
        x = int(float(final_list[i][0]))
        y = int(float(final_list[i][1]))
        center = (x,y)
        label = final_list[i][2]
        angle = int(float(final_list[i][3]))
        if angle < 0:
            angle = 360+angle
        width = int(final_list[i][4])
        height = int(final_list[i][5])
        size = (width, height)
        img_path = final_list[i][6]
        basename = os.path.basename(img_path)
        final_lst.append([center, label, angle, size, basename])
    return final_lst
        

In [12]:
folder = '/Users/malha/desktop/640x640/'
txt_file = '/Users/malha/desktop/640x640/SoyaImage3 (1).txt'
img_path = '/Users/malha/desktop/640x640/SoyaImage3 (1).jpg'
final_lst= get_center_label_size(txt_file)
final_lst

[[(378, 294), 'damaged', 327, (103, 78), 'Damaged_57.png'],
 [(564, 620), 'broken', 0, (86, 60), 'Broken_2.png'],
 [(448, 212), 'damaged', 231, (96, 101), 'Damaged_35.png'],
 [(377, 635), 'foreign', 181, (25, 29), 'Foreign_19.png'],
 [(202, 532), 'green', 358, (80, 100), 'Green_15.png'],
 [(316, 323), 'foreign', 327, (33, 102), 'Foreign_59.png'],
 [(187, 245), 'foreign', 33, (23, 101), 'Foreign_14.png'],
 [(339, 246), 'broken', 327, (89, 45), 'Broken_45.png'],
 [(568, 550), 'broken', 0, (96, 83), 'Broken_101.png'],
 [(44, 516), 'broken', 77, (84, 92), 'Broken_73.png'],
 [(339, 579), 'foreign', 0, (30, 141), 'Foreign_44.png'],
 [(409, 565), 'green', 40, (75, 82), 'Green_3.png'],
 [(280, 214), 'damaged', 237, (74, 88), 'Damaged_23.png'],
 [(284, 537), 'broken', 0, (78, 87), 'Broken_95.png'],
 [(374, 512), 'foreign', 82, (28, 40), 'Foreign_58.png'],
 [(197, 427), 'normal', 356, (83, 108), 'Normal_54.png'],
 [(473, 483), 'damaged', 316, (121, 72), 'Damaged_32.png'],
 [(453, 339), 'damaged'

In [13]:
def get_basepath(path):
    if 'Broken' in path:
        basepath = '/Users/malha/desktop/reimage/pic/Broken'
        filepath = os.path.join(basepath, path)
    if 'Damaged' in path:
        basepath = '/Users/malha/desktop/reimage/pic/Damaged'
        filepath = os.path.join(basepath, path)
    if 'Foreign' in path:
        basepath = '/Users/malha/desktop/reimage/pic/Foreign'
        filepath = os.path.join(basepath, path)
    if 'Green' in path:
        basepath = '/Users/malha/desktop/reimage/pic/Green'
        filepath = os.path.join(basepath, path)
    if 'Normal' in path:
        basepath = '/Users/malha/desktop/reimage/pic/Normal'
        filepath = os.path.join(basepath, path)
#     if 'twigs' in path:
#         basepath = '/Users/charissayu/desktop/reimage (3)/pic/twigs'
#         filepath = os.path.join(basepath, path)
    return filepath


In [14]:
final_lst

[[(378, 294), 'damaged', 327, (103, 78), 'Damaged_57.png'],
 [(564, 620), 'broken', 0, (86, 60), 'Broken_2.png'],
 [(448, 212), 'damaged', 231, (96, 101), 'Damaged_35.png'],
 [(377, 635), 'foreign', 181, (25, 29), 'Foreign_19.png'],
 [(202, 532), 'green', 358, (80, 100), 'Green_15.png'],
 [(316, 323), 'foreign', 327, (33, 102), 'Foreign_59.png'],
 [(187, 245), 'foreign', 33, (23, 101), 'Foreign_14.png'],
 [(339, 246), 'broken', 327, (89, 45), 'Broken_45.png'],
 [(568, 550), 'broken', 0, (96, 83), 'Broken_101.png'],
 [(44, 516), 'broken', 77, (84, 92), 'Broken_73.png'],
 [(339, 579), 'foreign', 0, (30, 141), 'Foreign_44.png'],
 [(409, 565), 'green', 40, (75, 82), 'Green_3.png'],
 [(280, 214), 'damaged', 237, (74, 88), 'Damaged_23.png'],
 [(284, 537), 'broken', 0, (78, 87), 'Broken_95.png'],
 [(374, 512), 'foreign', 82, (28, 40), 'Foreign_58.png'],
 [(197, 427), 'normal', 356, (83, 108), 'Normal_54.png'],
 [(473, 483), 'damaged', 316, (121, 72), 'Damaged_32.png'],
 [(453, 339), 'damaged'

In [15]:
object_list = []
for i in range(len(final_lst)):
    center_2 = final_lst[i][0]
    label = final_lst[i][1]
    angle = final_lst[i][2]
    size = final_lst[i][3]
    path = final_lst[i][4]
    
    df_1 = df.loc[df['path'] == path]
    
    contour = df_1['contour']
    center_0 = df_1['center_0']
    
    filepath = get_basepath(path)
    contours, cnt_rotated, center_1 = rotate_from_origin(filepath, angle)
    distance_x, distance_y = get_move_distance(center_1, center_2)
    
    cnt_rotated_move, xmin, ymin, xmax, ymax = move_contour_get_bbox(cnt_rotated, distance_x, distance_y)
    
    if ((xmax-xmin)<5) or ((ymax-ymin) < 5):
        print('i', i, 'xmax-xmin', xmax-xmin, 'ymax-ymin', ymax-ymin)
        print (final_lst[i])
        im = cv2.imread('/Users/malha/desktop/640x640/SoyaImage3 (1).jpg')
        im_copy = im.copy()

        cv2.drawContours(im_copy, contours, 0, (255, 0, 0), 1)
        cv2.drawContours(im_copy, [cnt_rotated], 0, (0, 255, 0), 1)
        cv2.drawContours(im_copy, [cnt_rotated_move], 0, (0, 255, 255), 1)
        cv2.rectangle(im_copy,(xmin,ymin),(xmax,ymax),(0,255,255), 1)
        plt.figure(figsize=(15,15))
        plt.imshow(im_copy)
        plt.axis("off")
    
    
    object_list.append([label, [xmin, ymin, xmax, ymax]])


In [16]:
def get_object_list(final_lst):
    object_list = []
    for i in range(len(final_lst)):
        center_2 = final_lst[i][0]
        label = final_lst[i][1]
        angle = final_lst[i][2]
        size = final_lst[i][3]
        path = final_lst[i][4]
        df_1 = df.loc[df['path'] == path]
        contour = df_1['contour']
        center_0 = df_1['center_0']
        filepath = get_basepath(path)
        contours, cnt_rotated, center_1 = rotate_from_origin(filepath, angle)
        distance_x, distance_y = get_move_distance(center_1, center_2)
        cnt_rotated_move, xmin, ymin, xmax, ymax = move_contour_get_bbox(cnt_rotated, distance_x, distance_y)
        object_list.append([label, [xmin, ymin, xmax, ymax]])
    return object_list

In [17]:
base_folder = '/Users/malha/desktop/640x640'
item_list = os.listdir('/Users/malha/desktop/640x640')
png_files = []
txt_files = []
for i in range(len(item_list)):
    if 'txt' in item_list[i]:
        file = os.path.join(base_folder, item_list[i])
        txt_files.append(file) 
    if 'png' in item_list[i]:
        file = os.path.join(base_folder, item_list[i])
        png_files.append(file)     

In [18]:
for i in range(len(txt_files)):
    txt_file = txt_files[i]
    img_path = txt_files[i][:-4]+'.png'
    final_lst = get_center_label_size(txt_file)
    object_list = get_object_list(final_lst)
    
    directory = base_folder
    imgfilepath = img_path
    object_list = object_list
    size_tuple = (640, 640, 3)
    savedir = base_folder
    
    create_xml(directory, imgfilepath, object_list, size_tuple, savedir)